# Import All Libraries to be used

In [12]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
print('here')
import torchvision.transforms as transforms
print('here2')
import numpy as np
import os
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import time
# from google.colab import drive
# drive.mount('/content/drive/') # mount your drive
# %cd '/content/drive/My Drive/Notability/AI6103 Deep Learning & Applications/assignment-mobilenet-code/'



here
here2


# Define plot_loss_acc

In [3]:
def plot_loss_acc(train_loss, val_loss, train_acc, val_acc, fig_name):
    x = np.arange(len(train_loss))
    max_loss = max(max(train_loss), max(val_loss))

    fig, ax1 = plt.subplots()
    ax1.set_xlabel('epoch')
    ax1.set_ylabel('loss')
    ax1.set_ylim([0,max_loss+1])
    lns1 = ax1.plot(x, train_loss, 'yo-', label='train_loss')
    lns2 = ax1.plot(x, val_loss, 'go-', label='val_loss')
    # ax1.tick_params(axis='y', labelcolor='tab:blue')

    ax2 = ax1.twinx()
    ax2.set_ylabel('accuracy')
    ax2.set_ylim([0,1])
    lns3 = ax2.plot(x, train_acc, 'bo-', label='train_acc')
    lns4 = ax2.plot(x, val_acc, 'ro-', label='val_acc')
    # ax2.tick_params(axis='y', labelcolor='tab:red')

    lns = lns1+lns2+lns3+lns4
    labs = [l.get_label() for l in lns]
    ax2.legend(lns, labs, loc=0)

    fig.tight_layout()
    plt.title(fig_name)

    plt.savefig('./diagram/' + fig_name)

    np.savez('./diagram/' + fig_name.replace('.png ', '.npz'), train_loss=train_loss, val_loss=val_loss, train_acc=train_acc, val_acc=val_acc)


# MobileNet

In [4]:
'''MobileNet in PyTorch.
See the paper "MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications"
for more details.
'''

class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), 1024]

    def __init__(self, num_classes=100):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32)
        self.linear = nn.Linear(1024, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x if isinstance(x, int) else x[0]
            stride = 1 if isinstance(x, int) else x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# Main (from .py, DO NOT RUN)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
# from mobilenet import MobileNet
# from utils import plot_loss_acc

def main(args):
    # fix random seeds
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    torch.use_deterministic_algorithms(True)

    # train val test
    # AI6103 students: You need to create the dataloaders youself
    train_loader, valid_loader = get_train_valid_loader(args.dataset_dir, args.batch_size, True, args.seed, save_images=args.save_images) 
    if args.test:
        test_loader = get_test_loader(args.dataset_dir, args.batch_size)

    # model
    model = MobileNet(100)
    print(model)
    model.cuda()

    # criterion
    criterion = torch.nn.CrossEntropyLoss().cuda()

    # optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=args.wd)
    if args.lr_scheduler:
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs)
    else:
        scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=1.0, total_iters=args.epochs)

    stat_training_loss = []
    stat_val_loss = []
    stat_training_acc = []
    stat_val_acc = []
    for epoch in range(args.epochs):
        training_loss = 0
        training_acc = 0
        training_samples = 0
        val_loss = 0
        val_acc = 0
        val_samples = 0
        # training
        model.train()
        for imgs, labels in train_loader:
            imgs = imgs.cuda()
            labels = labels.cuda()
           
            batch_size = imgs.shape[0]
            optimizer.zero_grad()
            logits = model.forward(imgs)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            _, top_class = logits.topk(1, dim=1)
            training_acc += torch.sum(equals.type(torch.FloatTensor)).item()
            training_loss += batch_size * loss.item()
            training_samples += batch_size
        # validation
        model.eval()
        for val_imgs, val_labels in valid_loader:
            batch_size = val_imgs.shape[0]
            val_logits = model.forward(val_imgs.cuda())
            loss = criterion(val_logits, val_labels.cuda())
            _, top_class = val_logits.topk(1, dim=1)
            equals = top_class == val_labels.cuda().view(*top_class.shape)
            val_acc += torch.sum(equals.type(torch.FloatTensor)).item()
            val_loss += batch_size * loss.item()
            val_samples += batch_size
        assert val_samples == 10000
        # update stats
        stat_training_loss.append(training_loss/training_samples)
        stat_val_loss.append(val_loss/val_samples)
        stat_training_acc.append(training_acc/training_samples)
        stat_val_acc.append(val_acc/val_samples)
        # print
        print(f"Epoch {(epoch+1):d}/{args.epochs:d}.. Learning rate: {scheduler.get_lr()[0]:.4f}.. Train loss: {(training_loss/training_samples):.4f}.. Train acc: {(training_acc/training_samples):.4f}.. Val loss: {(val_loss/val_samples):.4f}.. Val acc: {(val_acc/val_samples):.4f}")
        # lr scheduler
        scheduler.step()
    # plot
    plot_loss_acc(stat_training_loss, stat_val_loss, stat_training_acc, stat_val_acc, args.fig_name)
    # test
    if args.test:
        test_loss = 0
        test_acc = 0
        test_samples = 0
        for test_imgs, test_labels in test_loader:
            batch_size = test_imgs.shape[0]
            test_logits = model.forward(test_imgs.cuda())
            test_loss = criterion(test_logits, test_labels.cuda())
            _, top_class = test_logits.topk(1, dim=1)
            equals = top_class == test_labels.cuda().view(*top_class.shape)
            test_acc += torch.sum(equals.type(torch.FloatTensor)).item()
            test_loss += batch_size * test_loss.item()
            test_samples += batch_size
        assert test_samples == 10000
        print('Test loss: ', test_loss/test_samples)
        print('Test acc: ', test_acc/test_samples)



if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser(description='parameters')
    parser.add_argument('--dataset_dir',type=str, help='')
    parser.add_argument('--batch_size',type=int, help='')
    parser.add_argument('--epochs', type=int, help='')
    parser.add_argument('--lr',type=float, help='')
    parser.add_argument('--wd',type=float, help='')
    parser.add_argument('--fig_name',type=str, help='')
    parser.add_argument('--lr_scheduler', action='store_true')
    parser.set_defaults(lr_scheduler=False)
    parser.add_argument('--mixup', action='store_true')
    parser.set_defaults(mixup=False)
    parser.add_argument('--test', action='store_true')
    parser.set_defaults(test=False)
    parser.add_argument('--save_images', action='store_true')
    parser.set_defaults(save_images=False)
    parser.add_argument('--seed', type=int, default=0, help='')
    args = parser.parse_args()
    print(args)
    main(args)


# run (from .py, DO NOT RUN)



export CUBLAS_WORKSPACE_CONFIG=:16:8

python main.py \
--dataset_dir ./datasets \
--batch_size 128 \
--epochs 300 \
--lr 0.05 --wd 0.0005 \
--lr_scheduler \
--mixup \
--seed 0 \
--fig_name lr=0.05-lr_sche-wd=0.0005-mixup.png \
--test


# Parameters

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

cpu


# Data Preprocessing


In [6]:
# Download dataset
train = torchvision.datasets.CIFAR100(root='./temp', train=True, download=True, transform=transforms.ToTensor())
test = torchvision.datasets.CIFAR100(root='./temp', train=False, download=True, transform=transforms.ToTensor())
print(f'Size of train dataset downloaded: {len(train)}')
print(f'Size of test dataset downloaded: {len(test)}')

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./temp/cifar-100-python.tar.gz to ./temp
Files already downloaded and verified
Size of train dataset downloaded: 50000
Size of test dataset downloaded: 10000


In [7]:
# Randomly split downloaded training dataset into train_set and val_set with seed 0
train_set, val_set = torch.utils.data.random_split(train, [40000, 10000], generator=torch.Generator().manual_seed(0))
print(f'Size of train_set: {len(train_set)}')
print(f'Size of val_set: {len(val_set)}')

Size of train_set: 40000
Size of val_set: 10000


In [8]:
# Proportion of each class in train_set
print(type(train_set))

Count = {}
for i in range(len(train_set)):
  _, label = train_set[i]
  if label in Count:
    Count[label] += 1
  else:
    Count[label] = 1

plt.figure(figsize=[20,10])
plt.bar(Count.keys(), Count.values())
print(sum(Count.values()))

<class 'torch.utils.data.dataset.Subset'>
40000


In [9]:
# Compute mean and standard deviation
img, label = train_set[0]
img.size() # RGB, H, W

total = torch.tensor([0.0, 0.0, 0.0])
total_sq = torch.tensor([0.0, 0.0, 0.0])

img_h = img.size()[1]
img_w = img.size()[2]
pixel_count = len(train_set) * img_h * img_w

for i in range(len(train_set)):
  img, _ = train_set[i]
  total += img.sum(axis=[1,2])
  total_sq += (img**2).sum(axis=[1,2])

total_mean = total / pixel_count
total_std = torch.sqrt((total_sq/pixel_count) - total_mean**2)

print(f'mean: {total_mean}')
print(f'standard deviation: {total_std}')

mean: tensor([0.5068, 0.4861, 0.4403])
standard deviation: tensor([0.2671, 0.2563, 0.2759])


In [10]:
# Normalize data
transform_set = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(total_mean, total_std),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop((32, 32), padding=4)
])

train_norm = torchvision.datasets.CIFAR100(root='./temp', train=True, download=True, transform=transform_set)
train_set_norm, val_set_norm = torch.utils.data.random_split(train_norm, [40000, 10000], generator=torch.Generator().manual_seed(0))

total_norm = torch.tensor([0.0, 0.0, 0.0])
total_sq_norm = torch.tensor([0.0, 0.0, 0.0])

for i in range(len(train_set_norm)):
  img, _ = train_set_norm[i]
  total_norm += img.sum(axis=[1,2])
  total_sq_norm += (img**2).sum(axis=[1,2])

total_mean_norm = total_norm / pixel_count
total_std_norm = torch.sqrt((total_sq_norm/pixel_count) - total_mean_norm**2)

print(f'mean: {total_mean_norm}')
print(f'standard deviation: {total_std_norm}')

Files already downloaded and verified
mean: tensor([-0.0029, -0.0132, -0.0160])
standard deviation: tensor([0.9200, 0.9178, 0.9173])


# Show example pic

In [13]:
pic, label = (train_set[0])
print(pic.size())
plt.figure()
plt.imshow(pic.permute(1, 2, 0))
plt.show()
print(label)

torch.Size([3, 32, 32])
5


/var/folders/3l/hk29ld_90zqg6tqtpjmgnv7m0000gn/T/ipykernel_41960/1863608329.py:5: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [14]:
train_set[0][0]

tensor([[[0.4549, 0.4784, 0.5176,  ..., 0.7922, 0.7725, 0.7490],
         [0.4667, 0.4941, 0.5373,  ..., 0.7922, 0.7843, 0.7647],
         [0.4784, 0.5098, 0.5765,  ..., 0.7804, 0.7922, 0.7804],
         ...,
         [0.3216, 0.3373, 0.3490,  ..., 0.3176, 0.3059, 0.3020],
         [0.3255, 0.3412, 0.3490,  ..., 0.3176, 0.3059, 0.2863],
         [0.3216, 0.3373, 0.3569,  ..., 0.3216, 0.3020, 0.2745]],

        [[0.4078, 0.4235, 0.4706,  ..., 0.7294, 0.7137, 0.6941],
         [0.4235, 0.4392, 0.4745,  ..., 0.7255, 0.7255, 0.7137],
         [0.4353, 0.4510, 0.5020,  ..., 0.6941, 0.7255, 0.7294],
         ...,
         [0.2863, 0.3020, 0.3176,  ..., 0.2824, 0.2980, 0.2824],
         [0.2902, 0.3059, 0.3137,  ..., 0.2863, 0.2902, 0.2706],
         [0.2863, 0.3020, 0.3216,  ..., 0.2941, 0.2863, 0.2588]],

        [[0.3412, 0.3569, 0.3961,  ..., 0.6745, 0.6667, 0.6549],
         [0.3608, 0.3725, 0.3922,  ..., 0.6627, 0.6745, 0.6706],
         [0.3765, 0.3804, 0.4039,  ..., 0.6118, 0.6706, 0.

# Learning Rate

In [15]:
# Parameters for this portion
LR = [0.5, 0.05, 0.01]
BATCH_SIZE = 128
NUM_EPOCH = 15
WEIGHT_DECAY = 0
LR_SCHEDULE = None
SEED = 0
MOMENTUM = 0.9

# fix random seeds
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.use_deterministic_algorithms(False) # Originally True

# train val

train_loader = torch.utils.data.DataLoader(train_set_norm, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set_norm, batch_size=BATCH_SIZE, shuffle=True)

# criterion
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,12))
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
colors_train = ['ro-', 'go-', 'bo-']
colors_val = ['ro--', 'go--', 'bo--']

# loop for 3 different learning rates
for i in range(len(LR)):

  # model
  model = MobileNet(100)
  model = model.to(device)

  lr = LR[i]
  optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  if LR_SCHEDULE == True:
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, NUM_EPOCH)
  else:
    scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=1.0, total_iters=NUM_EPOCH)


  stat_training_loss = []
  stat_val_loss = []
  stat_training_acc = []
  stat_val_acc = []
  # equals = torch.tensor([0.0])
  for epoch in range(NUM_EPOCH):
    training_loss = 0
    training_acc = 0
    training_samples = 0
    val_loss = 0
    val_acc = 0
    val_samples = 0
    start_time = time.time()

    # training
    model.train()
    for imgs, labels in train_loader:
      imgs = imgs.to(device)
      labels = labels.to(device)
      
      batch_size = imgs.shape[0]
      optimizer.zero_grad()
      logits = model.forward(imgs)
      loss = criterion(logits, labels)
      loss.backward()
      optimizer.step()
      _, top_class = logits.topk(1, dim=1)
      equals = top_class == labels.view(*top_class.shape)
      training_acc += torch.sum(equals.type(torch.FloatTensor)).item()
      training_loss += batch_size * loss.item()
      training_samples += batch_size

    # validation
    model.eval()
    for val_imgs, val_labels in val_loader:
      val_imgs = val_imgs.to(device)
      val_labels = val_labels.to(device)

      batch_size = val_imgs.shape[0]
      val_logits = model.forward(val_imgs)
      loss = criterion(val_logits, val_labels)
      _, top_class = val_logits.topk(1, dim=1)
      equals = top_class == val_labels.view(*top_class.shape)
      val_acc += torch.sum(equals.type(torch.FloatTensor)).item()
      val_loss += batch_size * loss.item()
      val_samples += batch_size
    assert val_samples == 10000

    end_time = time.time()
    epoch_secs =  end_time - start_time

    # update stats
    stat_training_loss.append(training_loss/training_samples)
    stat_val_loss.append(val_loss/val_samples)
    stat_training_acc.append(training_acc/training_samples)
    stat_val_acc.append(val_acc/val_samples)
    # print
    print(f"Epoch {(epoch+1):d}/{NUM_EPOCH:d}, Epoch time: {epoch_secs:.2f}s Learning rate: {lr:.4f}, Train loss: {(training_loss/training_samples):.4f}, Train acc: {(training_acc/training_samples):.4f}, Val loss: {(val_loss/val_samples):.4f}, Val acc: {(val_acc/val_samples):.4f}")
    # lr scheduler
    scheduler.step()
  # plot
  # plot_loss_acc(stat_training_loss, stat_val_loss, stat_training_acc, stat_val_acc, "Task3_LearningRate.png")

  
  ax1.plot(stat_training_loss, colors_train[i], label='train_loss (lr = ' + str(lr) + ')')
  ax1.plot(stat_val_loss, colors_val[i], label='val_loss (lr = ' + str(lr) + ')')
  ax2.plot(stat_training_acc, colors_train[i], label='train_loss (lr = ' + str(lr) + ')')
  ax2.plot(stat_val_acc, colors_val[i], label='val_loss (lr = ' + str(lr) + ')')

ax1.legend()
ax2.legend()
plt.title("Task3_LearningRate")


Epoch 1/15, Epoch time: 209.48s Learning rate: 0.5000, Train loss: 4.5536, Train acc: 0.0199, Val loss: 4.3049, Val acc: 0.0287
Epoch 2/15, Epoch time: 234.34s Learning rate: 0.5000, Train loss: 4.1980, Train acc: 0.0404, Val loss: 4.0816, Val acc: 0.0624
Epoch 3/15, Epoch time: 229.38s Learning rate: 0.5000, Train loss: 3.9336, Train acc: 0.0764, Val loss: 3.8475, Val acc: 0.0868
Epoch 4/15, Epoch time: 243.18s Learning rate: 0.5000, Train loss: 3.7339, Train acc: 0.1069, Val loss: 3.6489, Val acc: 0.1258
Epoch 5/15, Epoch time: 222.93s Learning rate: 0.5000, Train loss: 3.4943, Train acc: 0.1485, Val loss: 3.6763, Val acc: 0.1398
Epoch 6/15, Epoch time: 222.99s Learning rate: 0.5000, Train loss: 3.2727, Train acc: 0.1875, Val loss: 3.2070, Val acc: 0.1986
Epoch 7/15, Epoch time: 215.29s Learning rate: 0.5000, Train loss: 3.0632, Train acc: 0.2223, Val loss: 3.0884, Val acc: 0.2237
Epoch 8/15, Epoch time: 223.48s Learning rate: 0.5000, Train loss: 2.8965, Train acc: 0.2553, Val loss: 

Text(0.5, 1.0, 'Task3_LearningRate')

In [18]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,12))
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
colors_train = ['ro-', 'go-', 'bo-']
colors_val = ['ro--', 'go--', 'bo--']

ax1.plot(stat_training_loss, colors_train[2], label='train_loss (lr = ' + str(lr) + ')')
ax1.plot(stat_val_loss, colors_val[2], label='val_loss (lr = ' + str(lr) + ')')
ax2.plot(stat_training_acc, colors_train[2], label='train_loss (lr = ' + str(lr) + ')')
ax2.plot(stat_val_acc, colors_val[2], label='val_loss (lr = ' + str(lr) + ')')


# ax1.legend()
# ax2.legend()

In [17]:
x = np.arange(15)
plt.plot(x, stat_training_loss)